# **Finding a suitable neighborhood**



## About the Project
**This project will deal with analyzing the neighborhoods between two cities, Toronto and New York City. A company named X wants to open a new branch in either Toronto or New York city. Considering the quality of life and also the optimum living standards for its employees, the company wants insights about the neighborhoods and local businesses in these cities. This project helps in determining which neighborhoods best fit the culture of the X company**

### Step 1 - Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# importing all necessary libraries for the project.
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
print('All necessary libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

**Downloading and cleaning the Toronto Dataset from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.**

The dataset is a list of Toronto's zipcodes which includes the names of neighborhoods and the boroughs

In [102]:
info = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#first table
toronto_df = info[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Cleaning and transforming the data available

In [103]:
toronto_df.rename(columns = {"Postal Code": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Ignore the cells which shows "Not assigned" in boroughs.
toronto_df.drop(toronto_df[toronto_df.Borough == 'Not assigned'].index, inplace=True)
toronto_df




,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [104]:
#Combining the neighborhoods that exists in one postal code
toronto_df = toronto_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#Change unassigned Neighborhood to its Borough's name
toronto_df.loc[85,'Neighborhood'] = 'Queen\'s Park'
print (toronto_df.shape)
toronto_df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Gathering the information of the longitudes and latitudes of the Neighborhoods of Toronto

In [105]:
latitude_longitude = pd.read_csv("http://cocl.us/Geospatial_data")
latitude_longitude.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [106]:
latitude_longitude.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latitude_longitude.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [107]:
latitude_longitude.tail()

,PostalCode,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [108]:
#Join the Latitude and Longitude dataframe to Neighborhoods dataframe
toronto_df.set_index("PostalCode")
latitude_longitude.set_index("PostalCode")
neighbor=pd.merge(toronto_df, latitude_longitude)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [109]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


In [110]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Canada are 43.6534817, -79.3839347.


In [12]:
# creating map of Toronto using folium library
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [111]:
scarborough_df = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [112]:
scarborough_df.shape

(17, 5)

In [113]:
address = 'Scarborough, Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Scarborough, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Scarborough, CA are 43.773077, -79.257774.


In [114]:
# creating map of Scarborough using Folium library 
scarborough_map = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(scarborough_df['Latitude'], scarborough_df['Longitude'], scarborough_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(scarborough_map)  
    
scarborough_map

## Using Foursquare API to explore the Scarborough neighborhood

In [115]:
CLIENT_ID = 'YGIWDSVTM2UK5APXCTSGJQWNRE5DBWCJHDPTWBR0WSK2WYWE' # Foursquare ID
CLIENT_SECRET = 'C4O1SZO3G0MBOXOSFLDRAUZABXR5D4ZPGBET2LQPN3HSVS52' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails of Foursquare:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails of Foursquare:
CLIENT_ID: YGIWDSVTM2UK5APXCTSGJQWNRE5DBWCJHDPTWBR0WSK2WYWE
CLIENT_SECRET:C4O1SZO3G0MBOXOSFLDRAUZABXR5D4ZPGBET2LQPN3HSVS52


## Defining a function to explore the neighborhoods

In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [117]:
venues_of_scarborough = getNearbyVenues(names=scarborough_df['Neighborhood'],
                                   latitudes=scarborough_df['Latitude'],
                                   longitudes=scarborough_df['Longitude']
                                  )

Found 89 venues in 17 neighborhoods.


In [118]:
print(venues_of_scarborough.shape)
venues_of_scarborough.head()


(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [119]:
#number of venues per each Neighborhood
venues_of_scarborough.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [120]:
print('There are {} distinct venues in {} categories.'.format(
    len(venues_of_scarborough['Venue'].unique()),len(venues_of_scarborough['Venue Category'].unique())))

There are 80 distinct venues in 55 categories.


## Using One hot encoding

In [121]:
OHE_of_scarborough = pd.get_dummies(venues_of_scarborough[['Venue Category']], prefix="", prefix_sep="")
# adding neighborhood column back to the dataframe
OHE_of_scarborough['Neighborhood'] = venues_of_scarborough['Neighborhood'] 
# moving neighborhood column to the first column in the dataframe
fixed_columns = [OHE_of_scarborough.columns[-1]] + list(OHE_of_scarborough.columns[:-1])
OHE_of_scarborough = OHE_of_scarborough[fixed_columns]
OHE_of_scarborough.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Grouping the rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [122]:
Grouped_scarborough = OHE_of_scarborough.groupby('Neighborhood').mean().reset_index()
Grouped_scarborough

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,0.000,0.000000,0.2,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.2,0.2,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.20,0.000000,0.0,0.000000,0.00,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.0,0.000000,0.25,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.0,0.000000,0.00,0.0
2,Cedarbrae,0.000000,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.000000,0.125,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.125000,0.00,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.0,0.0,0.00,0.000,0.083333,0.0,0.000000,0.00,0.0,0.00,0.000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.083333,0.083333,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.083333,0.000000,0.0,0.083333,0.166667,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.083333,0.00,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.5,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.200000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.400000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.1,0.000000,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.1,0.000,0.000000,0.0,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.125000,0.000000,0.000000,0.00,0.0,0.0,0.125,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.125,0.

## Showing each neighborhood with top 10 venues

In [123]:
num_top_venues = 10

for hood in Grouped_scarborough['Neighborhood']:
    print("----"+hood+"----")
    temp = Grouped_scarborough[Grouped_scarborough['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2
5          Accessories Store   0.0
6                       Park   0.0
7              Jewelry Store   0.0
8          Korean Restaurant   0.0
9             Medical Center   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink  0.25
1      General Entertainment  0.25
2                       Café  0.25
3            College Stadium  0.25
4          Accessories Store  0.00
5                       Park  0.00
6              Jewelry Store  0.00
7          Korean Restaurant  0.00
8  Latin American Restaurant  0.00
9                     Lounge  0.00


----Cedarbrae----
                  venue  freq
0    Athletics & Sports  0.12
1   Fried Chicken Joint  0.12
2                Bakery  0.12
3                  Bank  0.12
4   

## Sorting the venues in descending order

In [124]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Displaying top 10 venues for each neighborhood

In [125]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_sorted_venues = pd.DataFrame(columns=columns)
neighborhoods_sorted_venues['Neighborhood'] = Grouped_scarborough['Neighborhood']

for ind in np.arange(Grouped_scarborough.shape[0]):
    neighborhoods_sorted_venues.iloc[ind, 1:] = return_most_common_venues(Grouped_scarborough.iloc[ind, :], num_top_venues)

neighborhoods_sorted_venues

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Intersection,Bank,Italian Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,Bakery,American Restaurant,Grocery Store,General Entertainment,Gas Station
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Ice Cream Shop,Park,Bus Station,Metro Station,Intersection,Soccer Field,Fried Chicken Joint,Coffee Shop
7,"Guildwood, Morningside, West Hill",Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
8,"Kennedy Park, Ionview, East Birchmount Park",Train Station,Hobby Shop,Department Store,Coffee Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store


In [126]:
neighborhoods_sorted_venues.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                                        Bar
2nd Most Common Venue                 Construction & Landscaping
3rd Most Common Venue                      Vietnamese Restaurant
4th Most Common Venue                             Clothing Store
5th Most Common Venue                                        Gym
6th Most Common Venue                              Grocery Store
7th Most Common Venue                      General Entertainment
8th Most Common Venue                                Gas Station
9th Most Common Venue                        Fried Chicken Joint
10th Most Common Venue                      Fast Food Restaurant
Name: 11, dtype: object

## Using K-means to cluster the Scarborough neighborhood into 3 clusters

In [127]:
kclusters = 3
scarborough_grouped_clustering = Grouped_scarborough.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2], dtype=int32)

### Merging the clusters with the top 10 venues of the neighborhood 

In [128]:
#dropping the neighborhood Upper Rouge from the dataset since it does not have any values.
scarborough_df.drop(scarborough_df[scarborough_df.Neighborhood == 'Upper Rouge'].index, inplace = True)
merged_scarborough = scarborough_df
# adding clustering labels
merged_scarborough['Cluster Labels'] = kmeans.labels_
merged_scarborough= merged_scarborough.join(neighborhoods_sorted_venues.set_index('Neighborhood'), on='Neighborhood')
merged_scarborough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym


In [129]:
# creating map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_scarborough['Latitude'], merged_scarborough['Longitude'], merged_scarborough['Neighborhood'], merged_scarborough['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters of Scarborough neighborhood ( Cluster 0 , 1 , 2)

In [130]:
scarborough_cluster_0 = merged_scarborough.loc[merged_scarborough['Cluster Labels'] == 0, merged_scarborough.columns[[1] + list(range(4, merged_scarborough.shape[1]))]]

scarborough_cluster_1 = merged_scarborough.loc[merged_scarborough['Cluster Labels'] == 1, merged_scarborough.columns[[1] + list(range(4, merged_scarborough.shape[1]))]]

scarborough_cluster_2 = merged_scarborough.loc[merged_scarborough['Cluster Labels'] == 2, merged_scarborough.columns[[1] + list(range(4, merged_scarborough.shape[1]))]]

In [131]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,0,Accessories Store,Auto Garage,Bakery,Sandwich Place,Middle Eastern Restaurant,Smoke Shop,General Entertainment,Grocery Store,Clothing Store,Fried Chicken Joint


In [132]:
scarborough_cluster_1


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint


In [133]:
scarborough_cluster_2


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,Scarborough,-79.160497,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,-79.188711,2,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
3,Scarborough,-79.216917,2,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,-79.239476,2,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
5,Scarborough,-79.239476,2,Smoke Shop,Jewelry Store,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,Scarborough,-79.262029,2,Train Station,Hobby Shop,Department Store,Coffee Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,-79.284577,2,Bakery,Bus Line,Ice Cream Shop,Park,Bus Station,Metro Station,Intersection,Soccer Field,Fried Chicken Joint,Coffee Shop
8,Scarborough,-79.239476,2,Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,-79.273304,2,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,Bakery,American Restaurant,Grocery Store,General Entertainment,Gas Station


# Exploring the neighborhoods of New York City

###  I downloaded the data from  https://geo.nyu.edu/catalog/nyu_2451_34572 and placed it on the IBM server, so we can simply run a wget command and access the data

In [49]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')


Data downloaded!


In [50]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [51]:
df_neighborhoods = newyork_data['features']

In [52]:
df_neighborhoods[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [53]:
names_of_columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=names_of_columns)

In [54]:
for data in df_neighborhoods:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_latitude = neighborhood_latlon[1]
    neighborhood_longitude = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_latitude,
                                          'Longitude': neighborhood_longitude}, ignore_index=True)

In [55]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [56]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Geolibrary is used here to get the latitude and longitude of New York City

In [57]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [58]:
NewYork_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NewYork_map)  
    
NewYork_map


## Creating a new dataframe of Queens neighborhood

In [59]:
df_queens = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
df_queens.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [60]:
address = 'Queens, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [61]:
Queens_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_queens['Latitude'], df_queens['Longitude'], df_queens['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Queens_map)  
    
Queens_map

### getting the top hundred venues in Long Island City neighborhood within a radius of 500 meters

In [62]:
df_queens.loc[10, 'Neighborhood']

'Long Island City'

In [63]:
latitude_of_neighborhood = df_queens.loc[10, 'Latitude'] 
longitude_of_neighborhood = df_queens.loc[10, 'Longitude'] 
name_of_neighborhood = df_queens.loc[10, 'Neighborhood'] 
print('Latitude and longitude values of {} are {}, {}.'.format(name_of_neighborhood,
                                                               latitude_of_neighborhood, 
                                                               longitude_of_neighborhood))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


In [64]:
LIMIT = 100
radius = 500
url_foursquare = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_of_neighborhood, 
    longitude_of_neighborhood, 
    radius, 
    LIMIT)

url_foursquare

'https://api.foursquare.com/v2/venues/explore?&client_id=YGIWDSVTM2UK5APXCTSGJQWNRE5DBWCJHDPTWBR0WSK2WYWE&client_secret=C4O1SZO3G0MBOXOSFLDRAUZABXR5D4ZPGBET2LQPN3HSVS52&v=20180605&ll=40.75021734610528,-73.93920223915505&radius=500&limit=100'

In [65]:
#Send the GET request
results = requests.get(url_foursquare).json()

In [66]:
# to extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [67]:
venues = results['response']['groups'][0]['items']
venuesnearby = json_normalize(venues) # flatten JSON
columns_filtered = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venuesnearby =venuesnearby.loc[:, columns_filtered]
# filter the category for each row
venuesnearby['venue.categories'] = venuesnearby.apply(get_category_type, axis=1)
# cleaning columns
venuesnearby.columns = [col.split(".")[-1] for col in venuesnearby.columns]
venuesnearby.head()

,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Hilton Garden Inn,Hotel,40.750216,-73.936886
2,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
3,Dutch Kills,Cocktail Bar,40.747830,-73.940108
4,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010


In [68]:
print('{} venues were returned by Foursquare.'.format(venuesnearby.shape[0]))

71 venues were returned by Foursquare.


## Exploring neighborhoods in Queens

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

 

In [70]:
Venues_in_Queens = getNearbyVenues(names=df_queens['Neighborhood'],
                                   latitudes=df_queens['Latitude'],
                                   longitudes=df_queens['Longitude'])

Found 2103 venues in 81 neighborhoods.


In [72]:
Venues_in_Queens.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
2,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
3,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [73]:
print(Venues_in_Queens.shape)
Venues_in_Queens.tail()


(2103, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2098,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
2099,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
2100,Queensbridge,40.756091,-73.945631,Profundo Pool Club,40.753719,-73.948878,Hotel Pool
2101,Queensbridge,40.756091,-73.945631,Estate Garden And Grill,40.753700,-73.948841,Beer Garden
2102,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [74]:
#showing how many Venues are there per Neighborhood
Venues_in_Queens.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,20,20,20,20,20,20
Astoria,99,99,99,99,99,99
Astoria Heights,13,13,13,13,13,13
Auburndale,22,22,22,22,22,22
Bay Terrace,37,37,37,37,37,37
Bayside,72,72,72,72,72,72
Bayswater,1,1,1,1,1,1
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


In [75]:
print('There are {} distinct venues available in {} categories.'.format(
    len(Venues_in_Queens['Venue'].unique()),len(Venues_in_Queens['Venue Category'].unique())))

There are 1736 distinct venues available in 276 categories.


### One hot encoding

In [76]:
# one hot encoding
onehotencoding = pd.get_dummies(Venues_in_Queens[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
onehotencoding['Neighborhood'] = Venues_in_Queens['Neighborhood'] 
neighbor = onehotencoding['Neighborhood']
onehotencoding.drop(labels=['Neighborhood'], axis=1,inplace = True)
onehotencoding.insert(0, 'Neighborhood', neighbor)
onehotencoding.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Cente

### Grouping by Neighborhood and examining the frequency of the occurence of different venues

In [77]:
queens_neighborhood_grouped = onehotencoding.groupby('Neighborhood').mean().reset_index()
queens_neighborhood_grouped 

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Cente

### To Display each neighborhood with their respective top 5 venues

In [99]:
num_of_top_venues = 5

for hood in queens_neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_neighborhood_grouped[queens_neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_of_top_venues))
    print('\n')

----Arverne----
             venue  freq
0        Surf Spot  0.20
1    Metro Station  0.10
2   Sandwich Place  0.10
3    Burrito Place  0.05
4       Playground  0.05
5             Café  0.05
6       Board Shop  0.05
7  Thai Restaurant  0.05
8            Beach  0.05
9         Bus Stop  0.05


----Astoria----
                       venue  freq
0                        Bar  0.07
1  Middle Eastern Restaurant  0.06
2          Indian Restaurant  0.04
3         Seafood Restaurant  0.04
4                 Hookah Bar  0.04
5           Greek Restaurant  0.04
6                     Bakery  0.04
7   Mediterranean Restaurant  0.03
8                       Café  0.03
9            Bubble Tea Shop  0.02


----Astoria Heights----
                venue  freq
0        Burger Joint  0.07
1          Playground  0.07
2         Supermarket  0.07
3       Bowling Alley  0.07
4        Gourmet Shop  0.07
5  Chinese Restaurant  0.07
6        Cocktail Bar  0.07
7         Pizza Place  0.07
8              Bakery  0.07


In [78]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_of_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_of_top_venues]

### To show top 10 most common venues for each neighborhood

In [79]:
num_of_top_venues = 10

indicators = ['st', 'nd', 'rd']
# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_of_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

sorted_neighborhoods_venues = pd.DataFrame(columns=columns)
sorted_neighborhoods_venues['Neighborhood'] = queens_neighborhood_grouped['Neighborhood']

for ind in np.arange(queens_neighborhood_grouped.shape[0]):
    sorted_neighborhoods_venues.iloc[ind, 1:] = return_most_common_venues(queens_neighborhood_grouped.iloc[ind, :], num_of_top_venues)

sorted_neighborhoods_venues

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Restaurant,Bus Stop,Playground,Café,Board Shop,Beach,Thai Restaurant
1,Astoria,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Greek Restaurant,Bakery,Indian Restaurant
2,Astoria Heights,Bus Station,Italian Restaurant,Laundromat,Bakery,Bowling Alley,Supermarket,Food,Chinese Restaurant,Pizza Place,Hostel
3,Auburndale,Italian Restaurant,Mattress Store,Korean Restaurant,Bar,Furniture / Home Store,Gymnastics Gym,Toy / Game Store,Noodle House,Train,Athletics & Sports
4,Bay Terrace,Clothing Store,Shoe Store,Kids Store,American Restaurant,Mobile Phone Shop,Cosmetics Shop,Donut Shop,Women's Store,Supermarket,Gift Shop
5,Bayside,Bar,Pizza Place,Chinese Restaurant,Sushi Restaurant,Indian Restaurant,Ice Cream Shop,Spa,Italian Restaurant,Burger Joint,Bakery
6,Bayswater,Playground,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market
7,Beechhurst,Chinese Restaurant,Convenience Store,Gym,Dessert Shop,Pizza Place,Donut Shop,Park,Optical Shop,Shopping Mall,Shopping Plaza
8,Bellaire,Deli / Bodega,Halal Restaurant,Moving Target,Breakfast Spot,Chinese Restaurant,Greek Restaurant,Grocery Store,Bus Station,Coffee Shop,Gym
9,Belle Harbor,Beach,Deli / Bodega,Pub,Spa,Chinese Restaurant,Italian Restaurant,Pharmacy,Mexican Restaurant,Donut Shop,Boutique


In [80]:
sorted_neighborhoods_venues.iloc[47,]

Neighborhood                                      Long Island City
1st Most Common Venue                                        Hotel
2nd Most Common Venue                                  Coffee Shop
3rd Most Common Venue                                  Pizza Place
4th Most Common Venue                                          Bar
5th Most Common Venue                                         Café
6th Most Common Venue                           Mexican Restaurant
7th Most Common Venue                                   Donut Shop
8th Most Common Venue                         Gym / Fitness Center
9th Most Common Venue                                  Post Office
10th Most Common Venue    Residential Building (Apartment / Condo)
Name: 47, dtype: object

## Using K-Means to cluster the Queens Borough into 5 clusters

In [81]:
kclusters = 5
Clustering_queens_neighborhood_grouped = queens_neighborhood_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(Clustering_queens_neighborhood_grouped)
kmeans.labels_

array([4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 3, 1, 4, 1, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4], dtype=int32)

In [82]:
merged_queens = df_queens
merged_queens['Cluster Labels'] = kmeans.labels_
merged_queens = merged_queens.join(sorted_neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')
merged_queens.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,4,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Greek Restaurant,Bakery,Indian Restaurant
1,Queens,Woodside,40.746349,-73.901842,4,Grocery Store,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Bakery,Donut Shop,American Restaurant,Bar,Pub,Pizza Place
2,Queens,Jackson Heights,40.751981,-73.882821,4,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Grocery Store,Mexican Restaurant,Thai Restaurant,Pizza Place,Pharmacy
3,Queens,Elmhurst,40.744049,-73.881656,4,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,Vietnamese Restaurant,Chinese Restaurant,South American Restaurant,Bubble Tea Shop,Argentinian Restaurant,Bar,Park
4,Queens,Howard Beach,40.654225,-73.838138,4,Pharmacy,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Deli / Bodega,Clothing Store,Gym,Tapas Restaurant,Shipping Store,Sushi Restaurant


In [83]:
# create map
Clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(merged_queens['Latitude'], merged_queens['Longitude'], merged_queens['Neighborhood'], merged_queens['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Clusters_map)
       
Clusters_map

## Examining the clusters of Queens neighborhood ( Cluster 0 , 1 , 2 , 3 , 4)

In [85]:
queens_cluster_0 = merged_queens.loc[merged_queens['Cluster Labels'] == 0, merged_queens.columns[[1] + list(range(4, merged_queens.shape[1]))]]

queens_cluster_1 = merged_queens.loc[merged_queens['Cluster Labels'] == 1, merged_queens.columns[[1] + list(range(4, merged_queens.shape[1]))]]

queens_cluster_2 = merged_queens.loc[merged_queens['Cluster Labels'] == 2, merged_queens.columns[[1] + list(range(4, merged_queens.shape[1]))]]

queens_cluster_3 = merged_queens.loc[merged_queens['Cluster Labels'] == 3, merged_queens.columns[[1] + list(range(4, merged_queens.shape[1]))]]

queens_cluster_4 = merged_queens.loc[merged_queens['Cluster Labels'] == 4, merged_queens.columns[[1] + list(range(4, merged_queens.shape[1]))]]

In [86]:
queens_cluster_0


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Forest Hills,0,Gym / Fitness Center,Gym,Yoga Studio,Pizza Place,Convenience Store,Park,Thai Restaurant,Pharmacy,Optical Shop,Chinese Restaurant


In [87]:
queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Maspeth,1,Pizza Place,Diner,Chinese Restaurant,Grocery Store,Mobile Phone Shop,Bank,Park,Sandwich Place,Sushi Restaurant,Lounge
15,Glendale,1,Deli / Bodega,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Brewery,Food Stand,Food Court,Food Truck,Food,Eye Doctor
78,Hammels,1,Beach,Fried Chicken Joint,Gym / Fitness Center,Diner,Dog Run,Fast Food Restaurant,Shoe Store,Bus Stop,Bus Station,Food Truck


In [88]:
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,2,Deli / Bodega,Automotive Shop,Ice Cream Shop,History Museum,Spa,Grocery Store,Basketball Court,Korean Restaurant,Bakery,Donut Shop


In [89]:
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Elmhurst,3,Donut Shop,Gas Station,Bakery,Indie Movie Theater,Chinese Restaurant,American Restaurant,Coffee Shop,Hotel Bar,Ice Cream Shop,Rental Car Location
31,Jamaica Center,3,Pizza Place,Clothing Store,Mobile Phone Shop,Mexican Restaurant,Coffee Shop,Sandwich Place,Performing Arts Venue,Department Store,Caribbean Restaurant,Chinese Restaurant
52,Floral Park,3,Indian Restaurant,Pizza Place,Dosa Place,Salon / Barbershop,Basketball Court,Hobby Shop,Grocery Store,Fish Market,Fish & Chips Shop,Flea Market


In [90]:
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,4,Middle Eastern Restaurant,Bar,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Café,Greek Restaurant,Bakery,Indian Restaurant
1,Woodside,4,Grocery Store,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Bakery,Donut Shop,American Restaurant,Bar,Pub,Pizza Place
2,Jackson Heights,4,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Grocery Store,Mexican Restaurant,Thai Restaurant,Pizza Place,Pharmacy
3,Elmhurst,4,Thai Restaurant,Mexican Restaurant,Indonesian Restaurant,Vietnamese Restaurant,Chinese Restaurant,South American Restaurant,Bubble Tea Shop,Argentinian Restaurant,Bar,Park
4,Howard Beach,4,Pharmacy,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Deli / Bodega,Clothing Store,Gym,Tapas Restaurant,Shipping Store,Sushi Restaurant
5,Corona,4,Supermarket,Mexican Restaurant,South American Restaurant,Bakery,Restaurant,Park,Chinese Restaurant,Donut Shop,Basketball Court,Sandwich Place
7,Kew Gardens,4,Chinese Restaurant,Pizza Place,Bar,Bank,Donut Shop,Cosmetics Shop,Indian Restaurant,Park,Pet Store,Deli / Bodega
8,Richmond Hill,4,Latin American Restaurant,Pizza Place,Lounge,Bank,Clothing Store,Supermarket,Metro Station,Moving Target,Caribbean Restaurant,Spanish Restaurant
9,Flushing,4,Hotpot Restaurant,Korean Restaurant,Chinese Restaurant,Bubble Tea Shop,Construction & Landscaping,Karaoke Bar,Bakery,Asian Restaurant,Café,Gym / Fitness Center
10,Long Island City,4,Hotel,Coffee Shop,Pizza Place,Bar,Café,Mexican Restaurant,Donut Shop,Gym / Fitness Center,Post Office,Residential Building (Apartment / Condo)
